In [11]:
import joblib
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
model = joblib.load('../model/HAR_CNN.joblib')

In [6]:
label_mapping = {0: 'clapping', 1: 'dancing', 2: 'laughing', 3: 'running'}  # Update based on your label encoder

In [9]:
# Parameters
test_size = 0.2  # Percentage of data for the test set
val_size = 0.15   # Percentage of remaining data for validation set
random_seed = 42  # For reproducibility
img_size = (64, 64)  # Image dimensions
num_classes = 4  # Number of classes (e.g., clapping, dancing, laughing, running)
batch_size = 32

In [3]:
def preprocess_image(image_path):
    # Load the image in grayscale
    img = load_img(image_path, target_size=img_size, color_mode='grayscale')
    # Convert to array and normalize
    img_array = img_to_array(img) / 255.0
    # Add batch dimension
    return np.expand_dims(img_array, axis=0)

In [4]:
# Function to make prediction
def predict_action(image_path):
    #model = joblib.load('path/to/your/model') 
    # Preprocess the image
    preprocessed_img = preprocess_image(image_path)
    # Predict the class probabilities
    predictions = model.predict(preprocessed_img)
    # Get the class with the highest probability
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class_label = label_mapping[predicted_class_index]
    # Return the predicted label and probability
    return predicted_class_label, predictions[0][predicted_class_index]

In [13]:
# Example Usage
image_path = '../data/test-mg/Image_31.jpg'
predicted_label, confidence = predict_action(image_path)
print(f"Predicted Action: {predicted_label}")
print(f"Confidence: {confidence:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 701ms/step
Predicted Action: running
Confidence: 0.68


In [14]:
import tensorflow as tf

In [19]:

# Function to load the saved I3D model with appropriate tags
def load_i3d_model(model_dir, tags=['train']):
    """
    Load the I3D model with specified MetaGraph tags.
    - model_dir: Path to the directory containing the SavedModel.
    - tags: Tag set to specify the MetaGraph ('serve' for inference).
    """
    model = tf.compat.v1.saved_model.load_v2(model_dir, tags=tags)
    print(f"Model loaded successfully with tags: {tags}")
    return model

# Function to preprocess the input for I3D
def preprocess_input(video_frames, target_shape=(224, 224)):
    """
    Preprocess the input video frames.
    - Resize frames to the target shape.
    - Normalize pixel values to [0, 1].
    """
    preprocessed_frames = [
        tf.image.resize(frame, target_shape) / 255.0 for frame in video_frames
    ]
    return tf.convert_to_tensor(preprocessed_frames, dtype=tf.float32)

# Generate dummy input (replace this with actual video frames)
def generate_dummy_video(num_frames=32, height=224, width=224, channels=3):
    """
    Generate a dummy video with random pixel values.
    - num_frames: Number of frames in the video.
    - height, width: Frame dimensions.
    - channels: Number of channels (e.g., RGB).
    """
    return np.random.rand(num_frames, height, width, channels).astype(np.float32)

# Load the model
model_dir = "../model/i3d"  # Replace with the path where your saved_model.pb is located
i3d_model = load_i3d_model(model_dir, tags=['train'])  # Specify 'serve' tag for inference

# Prepare dummy video input (32 frames, 224x224 resolution, RGB)
#video_input = generate_dummy_video()

video_path = '../data/party1.mp4'
# Preprocess the input video
preprocessed_video = preprocess_input(video_input)

# Add batch dimension (batch_size=1)
input_tensor = tf.expand_dims(preprocessed_video, axis=0)

# Make a prediction
try:
    prediction = i3d_model.signatures["serving_default"](input_tensor)
    print("Prediction results:")
    print(prediction)
except Exception as e:
    print("Error during prediction:", str(e))


Model loaded successfully with tags: ['train']
Error during prediction: 'serving_default'


In [32]:
import cv2
import os

# Function to load class labels
def load_labels(label_file):
    """
    Load class labels from a label map file.
    - label_file: Path to the file containing labels.
    """
    with open(label_file, "r") as f:
        labels = [line.strip() for line in f.readlines()]
    return labels

# Assuming 'prediction' is the output from the model
def get_top_predictions(prediction, labels, top_k=5):
    """
    Map the model's numeric predictions to human-readable labels.
    - prediction: Model output array (e.g., logits or probabilities).
    - labels: List of class labels.
    - top_k: Number of top predictions to return.
    """
    # Convert predictions to probabilities if needed (e.g., softmax)
    probabilities = tf.nn.softmax(prediction).numpy()

    # Get top-k predictions
    top_indices = np.argsort(probabilities[0])[::-1][:top_k]
    top_predictions = [(labels[i], probabilities[0][i]) for i in top_indices]

    return top_predictions

# Function to preprocess input for I3D
def preprocess_input(video_frames, target_shape=(224, 224)):
    """
    Preprocess input video frames:
    - Resize frames to the target shape.
    - Normalize pixel values to [0, 1].
    """
    preprocessed_frames = [
        tf.image.resize(frame, target_shape) / 255.0 for frame in video_frames
    ]
    return tf.convert_to_tensor(preprocessed_frames, dtype=tf.float32)

# Function to load video file and convert to frames
def load_video(file_path, max_frames=32, resize_shape=(224, 224)):
    """
    Load a video from the file path, extract frames, and preprocess them.
    - file_path: Path to the video file.
    - max_frames: Maximum number of frames to load from the video.
    - resize_shape: Tuple specifying the frame dimensions (height, width).
    """
    cap = cv2.VideoCapture(file_path)
    frames = []
    frame_count = 0

    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        frames.append(frame)
        frame_count += 1

    cap.release()

    # Convert frames to tensor and preprocess
    frames_tensor = preprocess_input(frames, target_shape=resize_shape)
    return frames_tensor

# Load the model
def load_i3d_model(model_dir, tags={'train'}):
    """
    Load the I3D model with specified MetaGraph tags.
    - model_dir: Path to the directory containing the SavedModel.
    - tags: Tag set to specify the MetaGraph.
    """
    model = tf.compat.v1.saved_model.load_v2(model_dir, tags=tags)
    print(f"Model loaded successfully with tags: {tags}")
    return model

# Main script
model_dir = "../model/i3d/"  # Replace with your model directory path
video_path = "../data/party4.giff"  # Replace with your local video file path

# Load the model
i3d_model = load_i3d_model(model_dir, tags={'train'})

# Load and preprocess the video
video_tensor = load_video(video_path)

# Add batch dimension (batch_size=1)
input_tensor = tf.expand_dims(video_tensor, axis=0)

# Path to the label file
label_file = "../model/i3d/label_map.txt"  # Replace with the path to your labels file

# Load labels
labels = load_labels(label_file)

# Perform prediction
try:
    print("Available signatures:")
    print(i3d_model.signatures)

    # Example prediction (replace 'your_signature_key' with the correct key)
    signature_key = list(i3d_model.signatures.keys())[0]
    raw_prediction = i3d_model.signatures[signature_key](input_tensor)

    # Map predictions to class labels
    top_predictions = get_top_predictions(raw_prediction['default'], labels)
    print("Top Predictions:")
    for label, score in top_predictions:
        print(f"{label}: {score:.4f}")

except Exception as e:
    print("Error during prediction:", str(e))



Model loaded successfully with tags: {'train'}
Available signatures:
_SignatureMap({'default': <ConcreteFunction () -> Dict[['default', TensorSpec(shape=(None, 400), dtype=tf.float32, name=None)]] at 0x2698DC3E290>})
Top Predictions:
celebrating: 0.2188
dancing macarena: 0.0520
country line dancing: 0.0432
belly dancing: 0.0278
headbanging: 0.0271


In [ ]:
Model loaded successfully with tags: {'train'}
Available signatures:
_SignatureMap({'default': <ConcreteFunction () -> Dict[['default', TensorSpec(shape=(None, 400), dtype=tf.float32, name=None)]] at 0x269859F6290>})
Top Predictions:
celebrating: 0.2601
dancing macarena: 0.0443
pumping fist: 0.0297
headbanging: 0.0274
robot dancing: 0.0272

In [ ]:
celebrating: 0.1806
marching: 0.0548
dancing macarena: 0.0476
dancing gangnam style: 0.0379
pumping fist: 0.0325